In [46]:
import pandas as pd
import calendar

import glob
import os
import numpy as np

In [4]:
sangdo_number = 1120053

출력 결과물 형태 : 월, 요일, 유동인구 총합(일별)

이걸 만들어주는 함수를 만들어야 할듯 그 단계

1. 유동인구 값을 함유하는 모든 파일을 열 수 있게 그 이름들을 모두 리스트에 저장한다. (폴더 이름들도 저장한다.)
2. 폴더를 불러오면서, 해당 월의 값을 csv파일로 만든다. 
3. 각 요일 마다, 값을 합친다.

In [127]:

def monthly_flow(folder_path):
    # 파일이 있는 폴더 경로 설정


    # 폴더 안에 있는 모든 CSV 파일 경로 가져오기
    files = os.listdir(folder_path)

    # CSV 파일들을 담을 빈 리스트 생성
    dfs = []

    # 각 CSV 파일을 읽어서 데이터프레임으로 추가
    for file in files:
        if file.endswith('.csv'):
            file_path = os.path.join(folder_path, file)
            df = pd.read_csv(file_path,encoding='cp949')
            dfs.append(df)
    #print(dfs)
    # 모든 CSV 파일을 하나의 데이터프레임으로 병합
    combined_df = pd.concat(dfs, ignore_index=True)
    combined_df['이동인구(합)'] = combined_df['이동인구(합)'].replace('*', 2)



    combined_df = combined_df[(combined_df['도착 행정동 코드'] == sangdo_number) | (df['출발 행정동 코드'] == sangdo_number)]
    
    data_type = combined_df['이동인구(합)'].dtype
        # float 값을 int로 변환
    if np.issubdtype(data_type, np.floating):
        combined_df['이동인구(합)'] = combined_df['이동인구(합)'].astype(int)

    # str 값을 int로 변환
    if np.issubdtype(data_type, object):
        combined_df['이동인구(합)'] = pd.to_numeric(combined_df['이동인구(합)'], errors='coerce').astype(int)
    
    arrival_sums = combined_df.loc[combined_df['도착 행정동 코드'] == sangdo_number].groupby('요일')['이동인구(합)'].sum()

    # 요일 별로 '출발행정동'이 1120053인 경우 합계 값 계산
    departure_sums = combined_df.loc[combined_df['출발 행정동 코드'] == sangdo_number].groupby('요일')['이동인구(합)'].sum()
    
    # 대상연월 가져오기
    target_month = combined_df['대상연월'].unique()
    target_month = str(target_month[0])
    # 요일 가져오기
    days_of_week = combined_df['요일'].unique()

    results = arrival_sums.sub(departure_sums, fill_value=0)
        
    # 인덱스 초기화 
    results = results.reset_index()
    
    results['대상연월'] = target_month
    
    results['주중요일수'] = results['요일'].apply(lambda x: count_weekday(results['대상연월'].iloc[0], x))
    
    results['평균 이동인구'] = results['이동인구(합)']//results['주중요일수']
    
    return results



In [128]:
april = monthly_flow('/Users/hanyoungwook/2023-1/capstone_design/needed_data/LM2022/LM_202204')
print(april)

  요일  이동인구(합)    대상연월  주중요일수  평균 이동인구
0  금   258586  202204      5    51717
1  목   209803  202204      4    52450
2  수   206154  202204      4    51538
3  월   207481  202204      4    51870
4  일   158409  202204      4    39602
5  토   225248  202204      5    45049
6  화   209972  202204      4    52493


In [129]:
jan = monthly_flow('/Users/hanyoungwook/2023-1/capstone_design/needed_data/LM2022/LM_202201')
print(jan)

  요일  이동인구(합)    대상연월  주중요일수  평균 이동인구
0  금   178525  202201      4    44631
1  목   174911  202201      4    43727
2  수   171932  202201      4    42983
3  월   199291  202201      5    39858
4  일   170838  202201      5    34167
5  토   182166  202201      5    36433
6  화   173237  202201      4    43309


In [111]:
folder_path = '/Users/hanyoungwook/2023-1/capstone_design/needed_data/LM2022/LM_202204'

# 폴더 안에 있는 모든 CSV 파일 경로 가져오기
files = os.listdir(folder_path)

# CSV 파일들을 담을 빈 리스트 생성
dfs = []

# 각 CSV 파일을 읽어서 데이터프레임으로 추가
for file in files:
    if file.endswith('.csv'):
        file_path = os.path.join(folder_path, file)
        df = pd.read_csv(file_path,encoding='cp949')
        dfs.append(df)
#print(dfs)
# 모든 CSV 파일을 하나의 데이터프레임으로 병합
combined_df = pd.concat(dfs, ignore_index=True)
combined_df['이동인구(합)'] = combined_df['이동인구(합)'].replace('*', 2)



combined_df = combined_df[(combined_df['도착 행정동 코드'] == sangdo_number) | (df['출발 행정동 코드'] == sangdo_number)]

data_type = combined_df['이동인구(합)'].dtype
    # float 값을 int로 변환
if np.issubdtype(data_type, np.floating):
    combined_df['이동인구(합)'] = combined_df['이동인구(합)'].astype(int)

# str 값을 int로 변환
if np.issubdtype(data_type, object):
    combined_df['이동인구(합)'] = pd.to_numeric(combined_df['이동인구(합)'], errors='coerce').astype(int)

arrival_sums = combined_df.loc[combined_df['도착 행정동 코드'] == sangdo_number].groupby('요일')['이동인구(합)'].sum()

# 요일 별로 '출발행정동'이 1120053인 경우 합계 값 계산
departure_sums = combined_df.loc[combined_df['출발 행정동 코드'] == sangdo_number].groupby('요일')['이동인구(합)'].sum()

# 요일 별로 결과 값 계산
results = arrival_sums - departure_sums

# 대상연월 가져오기
target_month = combined_df['대상연월'].unique()

# 요일 가져오기
days_of_week = combined_df['요일'].unique()
# 데이터프레임 생성
#result_df = pd.DataFrame({'대상연월': target_month, '요일': days_of_week, '결과값': results}).reset_index(drop=True)



KeyboardInterrupt: 

In [114]:
target_month

'202204'

In [113]:
target_month = str(target_month[0])

In [107]:
print(departure_sums)
print(arrival_sums)
results = arrival_sums.sub(departure_sums, fill_value=0)
results



요일
금    71517
목    57958
수    58779
월    61664
일    38442
토    53654
화    60802
Name: 이동인구(합), dtype: int64
요일
금    330103
목    267761
수    264933
월    269145
일    196851
토    278902
화    270774
Name: 이동인구(합), dtype: int64


요일
금    258586
목    209803
수    206154
월    207481
일    158409
토    225248
화    209972
Name: 이동인구(합), dtype: int64

In [104]:
results

요일
금    258586
목    209803
수    206154
월    207481
일    158409
토    225248
화    209972
Name: 이동인구(합), dtype: int64

In [120]:
results['대상연월'] = '202204'

In [126]:
results

,요일,이동인구(합),대상연월,주중요일수,평균 이동인구
0,금,258586,202204,5,51717
1,목,209803,202204,4,52450
2,수,206154,202204,4,51538
3,월,207481,202204,4,51870
4,일,158409,202204,4,39602
5,토,225248,202204,5,45049
6,화,209972,202204,4,52493


In [118]:
results = results.reset_index()


In [122]:
results['주중요일수'] = results['요일'].apply(lambda x: count_weekday(results['대상연월'].iloc[0], x))


In [125]:
results['평균 이동인구'] = results['이동인구(합)']//results['주중요일수']

In [80]:

def count_weekday(target_month, target_weekday):
    year = int(target_month[:4])
    month = int(target_month[4:])
    
    _, last_day = calendar.monthrange(year, month)
    
    # 대상 월의 모든 날짜 생성
    dates = pd.date_range(start=f"{year}-{month:02d}-01", end=f"{year}-{month:02d}-{last_day}")
    
    # 한글 요일과 대응되는 영어 요일 매핑
    weekday_mapping = {
        '월': 'Monday',
        '화': 'Tuesday',
        '수': 'Wednesday',
        '목': 'Thursday',
        '금': 'Friday',
        '토': 'Saturday',
        '일': 'Sunday'
    }
    
    # 대상 요일에 해당하는 영어 요일 가져오기
    target_weekday_en = weekday_mapping.get(target_weekday)
    
    if target_weekday_en is None:
        raise ValueError("올바른 한글 요일을 입력하세요.")
    
    # 대상 요일과 일치하는 횟수 계산
    weekday_count = sum(1 for date in dates if date.strftime('%A') == target_weekday_en)
    
    return weekday_count



여기에 모든 파일을 열어서 합치는 과정을 거쳐야 한다.

In [132]:

def apply_monthly_flow_to_folders(folder_path):
    # 폴더 내에 있는 폴더 목록 가져오기
    folder_list = [folder_name for folder_name in os.listdir(folder_path) if not folder_name.startswith('.')]
    
    # 결과를 저장할 빈 리스트 초기화
    dfs = []
    
    # 각 폴더에 대해 monthly_flow 함수 적용
    for folder_name in folder_list:
        # 폴더 경로 생성
        subfolder_path = os.path.join(folder_path, folder_name)
        
        # monthly_flow 함수 적용
        result = monthly_flow(subfolder_path)
        
        # 결과를 데이터프레임으로 변환하여 리스트에 추가
        df = pd.DataFrame(result)
        dfs.append(df)
    
    # 결과 데이터프레임 생성
    results = pd.concat(dfs, ignore_index=True)
    
    return results



In [133]:
folder_path = '/Users/hanyoungwook/2023-1/capstone_design/needed_data/LM2022'
results = apply_monthly_flow_to_folders(folder_path)
print(results)

    요일  이동인구(합)    대상연월  주중요일수  평균 이동인구
0    금   178525  202201      4    44631
1    목   174911  202201      4    43727
2    수   171932  202201      4    42983
3    월   199291  202201      5    39858
4    일   170838  202201      5    34167
..  ..      ...     ...    ...      ...
100  수   291139  202211      5    58227
101  월   228616  202211      4    57154
102  일   172072  202211      4    43018
103  토   195147  202211      4    48786
104  화   292851  202211      5    58570

[105 rows x 5 columns]


In [134]:
results.to_csv('LM_sangdo', index=False)